In [ ]:
!pip install kaggle
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c shopee-product-detection-student
!unzip -q shopee-product-detection-student.zip

Uninstalling kaggle-1.5.9:
  Successfully uninstalled kaggle-1.5.9
     |████████████████████████████████| 1.5MB 7.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 4.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=432de7740951695c6788b951d1ff9d197caf3e0f0fe833aeb4fe38734103fef7
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
100% 9.37G/9.38G [03:38<00:00, 87.7MB/s]
100% 9.38G/9.38G [03:38<00:00, 46.0MB/s]


In [2]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy

class VGG19(Sequential):
  def __init__(self, input_shape,k):
    super().__init__()
    self.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape = input_shape))
    self.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(MaxPool2D(pool_size=(2,2), strides = (2,2)))

    self.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(MaxPool2D(pool_size=(2,2), strides = (2,2)))

    self.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(Conv2D(512, kernel_size=(3,3), padding='same', activation='relu'))
    self.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))

    self.add(Flatten())
    self.add(Dense(4096, activation='relu'))
    self.add(Dropout(0.5))
    self.add(Dense(4096, activation='relu'))
    self.add(Dropout(0.5))
    self.add(Dense(1000, activation='relu'))
    self.add(Dropout(0.5))
    self.add(Dense(k, activation = 'softmax'))

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D

from tensorflow.keras import backend as K


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def relu6(x):
    return K.relu(x, max_value=6.0)


def _conv_block(inputs, filters, kernel, strides):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)
    x = BatchNormalization(axis=channel_axis)(x)
    return Activation(relu6)(x)


def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    tchannel = K.int_shape(inputs)[channel_axis] * t
    cchannel = int(filters * alpha)

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation(relu6)(x)

    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization(axis=channel_axis)(x)

    if r:
        x = Add()([x, inputs])

    return x


def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    x = _bottleneck(inputs, filters, kernel, t, alpha, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)

    return x


def MobileNetv2(input_shape, k, alpha=1.0):
    inputs = Input(shape=input_shape)

    first_filters = _make_divisible(32 * alpha, 8)
    x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=1, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 24, (3, 3), t=6, alpha=alpha, strides=2, n=2)
    x = _inverted_residual_block(x, 32, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    x = _inverted_residual_block(x, 64, (3, 3), t=6, alpha=alpha, strides=2, n=4)
    x = _inverted_residual_block(x, 96, (3, 3), t=6, alpha=alpha, strides=1, n=3)
    x = _inverted_residual_block(x, 160, (3, 3), t=6, alpha=alpha, strides=2, n=3)
    x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    if alpha > 1.0:
        last_filters = _make_divisible(1280 * alpha, 8)
    else:
        last_filters = 1280

    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)

    return model



In [ ]:
model = MobileNetv2((32,32,3),1000)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
print(model.summary())

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

ds_train, ds_test = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = tf.keras.applications.MobileNetV2(
    input_shape=(32,32,3),
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=10,
    classifier_activation="softmax",
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

print(ds_train)
print(ds_test)
history = model.fit(
    ds_train,
    epochs=100,
    validation_data=ds_test
)
print(history)

model.save('/content/drive/My Drive/Shopee/mobilenetv2-keras-cifar10.hdf5')

<PrefetchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float32, tf.int64)>
Epoch 1/100
391/391 [==============================] - 16s 41ms/step - loss: nan - sparse_categorical_accuracy: 0.0100 - val_loss: nan - val_sparse_categorical_accuracy: 0.0100
Epoch 2/100
391/391 [==============================] - 9s 24ms/step - loss: nan - sparse_categorical_accuracy: 0.0100 - val_loss: nan - val_sparse_categorical_accuracy: 0.0100
Epoch 3/100
391/391 [==============================] - 9s 24ms/step - loss: nan - sparse_categorical_accuracy: 0.0100 - val_loss: nan - val_sparse_categorical_accuracy: 0.0100
Epoch 4/100
391/391 [==============================] - 9s 24ms/step - loss: nan - sparse_categorical_accuracy: 0.0100 - val_loss: nan - val_sparse_categorical_accuracy: 0.0100
Epoch 5/100
391/391 [==============================] - 9s 23ms/step - loss: nan - sparse_categorical_accuracy: 0.0100 - val

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

ds_train, ds_test = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = VGG19((32,32,3), 10)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test
)
print(history)

model.save('/content/drive/My Drive/Shopee/vgg19-cifar10.hdf5')

Epoch 1/5
  2/391 [..............................] - ETA: 14s - loss: 2.3025 - sparse_categorical_accuracy: 0.1484WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0241s vs `on_train_batch_end` time: 0.0480s). Check your callbacks.


391/391 [==============================] - 31s 79ms/step - loss: 2.3603 - sparse_categorical_accuracy: 0.0998 - val_loss: 2.3612 - val_sparse_categorical_accuracy: 0.1000
Epoch 2/5
391/391 [==============================] - 29s 74ms/step - loss: 2.3611 - sparse_categorical_accuracy: 0.1000 - val_loss: 2.3612 - val_sparse_categorical_accuracy: 0.1000
Epoch 3/5
391/391 [==============================] - 29s 74ms/step - loss: 2.3611 - sparse_categorical_accuracy: 0.1000 - val_loss: 2.3612 - val_sparse_categorical_accuracy: 0.1000
Epoch 4/5
391/391 [==============================] - 29s 75ms/step - loss: 2.3611 - sparse_categorical_accuracy: 0.1000 - val_loss: 2.3612 - val_sparse_categorical_accuracy: 0.1000
Epoch 5/5
391/391 [==============================] - 29s 74ms/step - loss: 2.3611 - sparse_categorical_accuracy: 0.1000 - val_loss: 2.3612 - val_sparse_categorical_accuracy: 0.1000


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

ds_train, ds_test = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = MobileNetv2((28,28,1), 10)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

print(ds_train)
print(ds_test)
history = model.fit(
    ds_train,
    epochs=100,
    validation_data=ds_test
)
print(history)

model.save('/content/drive/My Drive/Shopee/mobilenetv2-kmnist.hdf5')

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>
Epoch 1/100
469/469 [==============================] - 17s 36ms/step - loss: 1.8598 - sparse_categorical_accuracy: 0.5975 - val_loss: 2.3060 - val_sparse_categorical_accuracy: 0.1000
Epoch 2/100
469/469 [==============================] - 11s 23ms/step - loss: 1.6728 - sparse_categorical_accuracy: 0.7874 - val_loss: 1.8532 - val_sparse_categorical_accuracy: 0.6069
Epoch 3/100
469/469 [==============================] - 11s 23ms/step - loss: 1.6337 - sparse_categorical_accuracy: 0.8272 - val_loss: 1.7813 - val_sparse_categorical_accuracy: 0.6793
Epoch 4/100
469/469 [==============================] - 11s 23ms/step - loss: 1.6103 - sparse_categorical_accuracy: 0.8506 - val_loss: 1.7041 - val_sparse_categorical_accuracy: 0.7571
Epoch 5/100
469/469 [==============================] - 11s 23ms/step - loss: 1.5938 - sparse_cat

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

ds_train, ds_test = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

model = VGG19((28,28,1),10)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
history = model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test
)
print(history)

model.save('/content/drive/My Drive/Shopee/vgg19-kmnist.hdf5')

ValueError: ignored